In [41]:
from datetime import time

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_mantine_components as dmc

from Utils import City, AggregationLevel, TrafficType
from DataPreprocessing.WeatherData.Data import WeatherData
from DataPreprocessing.TrafficData.Data import CityTrafficData
from DataPreprocessing.GeoData.GeoData import IrisGeoData
from ExpectedBedTime.ExpectedBedTimeAPI import ExpectedBedTimeAPI
from DataPreprocessing.AdminData.AdminDataComplete import AdminData
from ExpectedBedTime.Plots import StartBedTimeRobustness
from FeatureSelection.BasicFeatureImportanceCalculator import BasicFeatureImportanceCalculator

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
iris_geo_data = IrisGeoData()

In [42]:
admin_data = AdminData()

In [52]:
admin_data.get_admin_data()

,P19_POP,P19_POP75P,P19_POP6074,P19_POP4559,P19_POP3044,P19_POP1529,P19_POP0002,P19_POP0305,P19_POP0610,P19_POP1114,...,EQUIP_D3,EQUIP_D4,EQUIP_D5,EQUIP_D6,EQUIP_D7,EQUIP_E1,EQUIP_F1,EQUIP_F2,EQUIP_F3,EQUIP_G1
iris,,,,,,,,,,,,,,,,,,,,,
010040101,1866.820612,145.944137,239.906368,210.738586,357.664830,593.876491,81.762602,47.104967,100.705691,89.116940,...,1.0,1.0,0.0,0.0,2.0,2.0,12.0,0.0,0.0,2.0
010040102,3597.838556,312.117891,512.225613,597.475022,718.946258,849.032268,168.897927,119.186603,198.365404,121.591571,...,2.0,0.0,0.0,3.0,1.0,4.0,3.0,0.0,0.0,0.0
010040201,4134.477890,436.875291,599.717316,634.759383,756.674840,877.610261,167.871616,174.765320,272.922347,213.281515,...,2.0,2.0,2.0,1.0,1.0,1.0,7.0,0.0,3.0,0.0
010040202,4534.862942,344.226989,702.290170,948.374118,897.112540,756.264583,149.032630,182.900634,328.935646,225.725632,...,1.0,0.0,1.0,0.0,3.0,2.0,7.0,0.0,0.0,1.0
010330102,2918.101931,326.046160,295.841308,465.734930,628.637419,673.928559,131.760371,71.511391,219.919760,104.722032,...,5.0,1.0,1.0,2.0,0.0,1.0,8.0,0.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974230201,781.576282,52.441247,91.772183,207.748018,170.434054,121.018263,25.212138,28.237595,42.356392,42.356392,...,0.0,0.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,0.0
974240101,1737.999244,148.717804,249.717518,405.379012,324.242397,246.910337,69.588242,58.784278,118.186797,116.472861,...,3.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0,1.0,9.0
974240102,1677.366109,82.874067,193.445907,435.660591,309.994224,287.116166,63.035316,66.841666,126.343354,112.054817,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [6]:
lyon_traffic_data = CityTrafficData(city=City.LYON, aggregation_level=AggregationLevel.IRIS, traffic_type=TrafficType.USERS)

100%|██████████| 77/77 [00:22<00:00,  3.49it/s]


In [7]:
expected_bed_time_lyon = ExpectedBedTimeAPI.compute_expected_bed_times(traffic_data=lyon_traffic_data)[0]

In [9]:
expected_bed_time_lyon.geo_plot(iris_geo_data=iris_geo_data, n_quantiles=5)
expected_bed_time_lyon.plot()

In [53]:
feature_selector = BasicFeatureImportanceCalculator(admin_data=admin_data, expected_bed_time=expected_bed_time_lyon)

In [55]:
feature_selector.compute_feature_importance()

,f_statistic,p_values,mutual_information,regression,description
P19_POP,2.134476,9.528897e-02,0.016831,0.110254,Population
P19_POP75P,4.870993,2.436819e-03,0.042771,-0.179381,Nombre de personnes de 75 ans ou plus
P19_POP6074,10.647999,9.399752e-07,0.030756,-0.273457,Nombre de personnes de 60 à 74 ans
P19_POP4559,2.513112,5.808348e-02,0.000000,-0.130662,Nombre de personnes de 45 à 59 ans
P19_POP3044,4.032395,7.606094e-03,0.002939,0.164598,Nombre de personnes de 30 à 44 ans
...,...,...,...,...,...
EQUIP_E1,2.817541,3.883921e-02,0.003198,0.144475,INFRASTRUCTURES DE TRANSPORTS
EQUIP_F1,4.883737,2.394925e-03,0.000000,-0.154807,ÉQUIPEMENTS SPORTIFS
EQUIP_F2,0.647312,5.850160e-01,0.000000,-0.067559,ÉQUIPEMENTS DE LOISIRS
EQUIP_F3,1.211892,3.050321e-01,0.000000,-0.028619,ÉQUIPEMENTS CULTURELS ET SOCIOCULTURELS


In [28]:
admin_data.metadata

,COD_VAR,LIB_VAR_LONG
0,C19_ACT1564,Nombre de personnes actives de 15 à 64 ans
1,C19_ACT1564_CS1,Nombre d'agriculteurs exploitants actifs de 15...
2,C19_ACT1564_CS2,"Nombre d'artisans, commerçants, chefs d'entrep..."
3,C19_ACT1564_CS3,Nombre de cadres et professions intellectuelle...
4,C19_ACT1564_CS4,Nombre de professions intermédiaires actifs de...
...,...,...
394,P19_SCOL1517,Nombre de personnes scolarisées de 15 à 17 ans
395,P19_SCOL1824,Nombre de personnes scolarisées de 18 à 24 ans
396,P19_SCOL2529,Nombre de personnes scolarisées de 25 à 29 ans
397,P19_SCOL30P,Nombre de personnes scolarisées de 30 ans ou plus


## Plots for the robustness to change in start_bed_time

In [197]:
start_bed_times = [time(21), time(21,15), time(21, 30), time(21,45), time(22), time(22,15), time(22,30)]
ExpectedBedTimeAPI.plot_start_bed_time_robustness(traffic_data=city_traffic_data, start_bed_times=start_bed_times)